# OpenGroup Biotope workshop

In this notebook, we will show how to download and visualise sensor data using the O-MI and O-DF standards. We will use the network of sensors deployed in Lyon as our case study.

We will be using python to download and process the data, and use the [vega lite](https://vega.github.io/) data visualisation grammar to create interactive charts.

## Step 1: initialisation

Let's start by importing the libraries that we will need in this project.

[Pandas](https://pandas.pydata.org/) is the well-known library that provides high performance data structure and data analysis tools.

[Altair](https://altair-viz.github.io/) allows us to generate vega lite charts via a python API.

In [ ]:
import requests # HTTP requests
from bs4 import BeautifulSoup # XML parser
import pandas as pd # data structure and processing
import altair as alt # data visualisation
import utils # utility functions developed for the workshop, see utils.py
from datetime import datetime
import json
alt.renderers.enable('notebook')

In [ ]:
# Some of the sensors are defective, we declare them here so we can exclude them from future processing
defective_sensors = ["Sensor:SL-T-P12", "Sensor:SL-T-P9", "Sensor:SL-T-P11", "Sensor:SL-T-G1"]

In [ ]:
# URL of the OM-I node
url = "https://biotope-omi.alpha.grandlyon.com/"

## Step 2: querying the O-MI node

The [O-MI web client](https://biotope-omi.alpha.grandlyon.com/html/webclient/index.html) allows you to easily discover the hierarchy in which the sensors are organised.

In this notebook, we will be using the temperature sensors located in:
- `Organization:Metropole-de-Lyon:v1-0-0/OrganizationalUnit:DINSI/Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:I`
- `Organization:Metropole-de-Lyon:v1-0-0/OrganizationalUnit:DINSI/Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:II`

Let's start by querying a single sensor. For example `Organization:Metropole-de-Lyon:v1-0-0/OrganizationalUnit:DINSI/Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:I/Sensor:SL-T-P1`. The function `utils.dictionary2omi` allows us to generate a O-MI query from a hierarchy described as a python dictionary.

In [ ]:
hierarchy = {
    "Organization:Metropole-de-Lyon:v1-0-0" : {
        "OrganizationalUnit:DINSI": {
            "Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:I": {
                "Sensor:SL-T-P1"
            }
        }
    }
}
xmlReq = utils.dictionary2omi(hierarchy)

In [ ]:
print(BeautifulSoup(xmlReq).prettify())

In [ ]:
# Send query to O-MI node
resp = requests.post(url, data=xmlReq)
# Parse response
xmlResp = BeautifulSoup(resp.text)

The XML response to the query contains a variety of information relating to the sensor and the owner of the sensor, as well as its last measurement.

In [ ]:
print(xmlResp.prettify())

We can query more than one measurement at once. For example, we can download the last 100 measurements using the `newest` parameter.

In [ ]:
hierarchy = {
    "Organization:Metropole-de-Lyon:v1-0-0" : {
        "OrganizationalUnit:DINSI": {
            "Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:I": {
                "Sensor:SL-T-P1"
            }
        }
    }
}
xmlReq = utils.dictionary2omi(hierarchy, newest=100)

In [ ]:
sltp1 = BeautifulSoup(requests.post(url, data=xmlReq).text)

In [ ]:
print(sltp1.prettify())

We can also query an object and all of its children.

In [ ]:
hierarchy = {
    "Organization:Metropole-de-Lyon:v1-0-0" : {
        "OrganizationalUnit:DINSI": {
            "Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:I",
            "Deployment:Sensing-Labs-IP68-Outdoor-Temperature-Sensor:II"
        }
    }
}
xmlReq = utils.dictionary2omi(hierarchy, newest=100)

In [ ]:
all_sensors = BeautifulSoup(requests.post(url, data=xmlReq).text)

In [ ]:
print(all_sensors.prettify())

## Step 3: parsing and visualising

Now that we have downloaded the raw data, we need to extract the information that we want to study.

Let's say we want to visualise a curve of the temperature of the last few days for the SL-T-P1 sensor. We need to extract the temperature and the timestamp from the XML.

In [ ]:
# Get all relevant "value" tags
values = sltp1.find('object', type="sosa:Observation").infoitem.find_all('value')

data = {
    "timestamp": [],
    "value": []
}
# Extract value and timestamp from the tags
for v in values:
    data["value"].append(float(v.text))
    data["timestamp"].append(v.get("datetime"))

# Store result in a pandas DataFrame
df = pd.DataFrame(data)

We can then chart the result using Altair.
```python
alt.Chart(df) # Chart the data contained in df
.mark_line()  # using a line chart
.encode(      # with the following properties
    x='timestamp:T', # x axis: data from the 'timestamp' column, type Time
    y='value:Q'      # y axis: data from the 'value' column, type Quantitative
)
```

In [ ]:
alt.Chart(df).mark_line().encode(
    x='timestamp:T',
    y='value:Q'
)

Let's move on to a more complex use case. This time, we want to display all the sensors on a map as points, and colour them depending on their last temperature measurement.

In [ ]:
# Parsing
lora = all_sensors.omienvelope.response.result.msg.objects.find_all('object', type="seas:LoRaCommunicationDevice")
data = {
    "name": [],
    "latitude": [],
    "longitude": [],
    "lastValue": []
}
for s in lora:
    name = s.id.string
    # ignore defective sensors
    if name in defective_sensors:
        continue    
    obs = s.find('object', type="sosa:Observation")
    data["name"].append(name)
    data["longitude"].append(float(s.find('infoitem', {"name":"geo:long"}).string))
    data["latitude"].append(float(s.find('infoitem', {"name":"geo:lat"}).string))
    data["lastValue"].append(float(obs.infoitem.find("value").string)) # We only use the first value
df = pd.DataFrame(data)

In [ ]:
# Data visualisation
# utils.geoChart is a custom function to automatically transform longitude/latitude fields
# into geographical point ready for plotting
chart = alt.Chart(df).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('lastValue:Q', scale=alt.Scale(scheme="inferno"))
)
chart

In [ ]:
# We can add the outline of the metropole to the chart to have a better understanding of the sensors' positions
file = open('grandlyon.json', encoding='utf-8')
geojson = json.load(file)
file.close()
metropole_outline = alt.Data(values=geojson, format=alt.JsonDataFormat(property='features'))
metropole = alt.Chart(metropole_outline).mark_geoshape(
    stroke='red'
).encode()
metropole + chart

# TODO: remove ?

In [ ]:
df2 = df[~df['name'].isin(["Sensor:SL-T-G9", "Sensor:SL-T-G10", "Sensor:SL-T-G11", "Sensor:SL-T-G12"])]
chart = alt.Chart(df2).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    #color=alt.Color('lastValue:Q', scale=alt.Scale(scheme="inferno"))
    color='name:N'
)
chart

We can also represent multiple sensors' measurements on the same chart using color coding.

In [ ]:
data = {
    "name": [],
    "timestamp": [],
    "value": []
}
sensors = xmlResp.omienvelope.response.result.msg.objects.find_all('object', type="seas:LoRaCommunicationDevice")
for s in lora:
    obs = s.find('object', type="sosa:Observation")
    name = s.id.string
    if name in defective_sensors:
        continue
    for v in obs.infoitem.find_all("value"):
        data["name"].append(name)
        data["timestamp"].append(v.get("datetime"))
        data["value"].append(float(v.string))
df = pd.DataFrame(data)

In [ ]:
alt.Chart(df).mark_line().encode(
    x=alt.X('timestamp:T'),
    y=alt.Y('value'),
    color='name'
)

## Exercise: bottle banks

Chart ideas:
- Map of bottle banks' capacity
- Fill rate evolution over time
- Current fill rate using bar charts (mark_bar)

In [ ]:
# Define query
hierarchy = {
    "Organization:SigrenEa-V1.1.0" : {
        "Deployment:Bottle_Bank:1edd171c-5f2d-11e8-a6ab-10604b7fb2e7"
    }
}
xmlReq = utils.dictionary2omi(hierarchy)

In [23]:
# Send query
resp = requests.post(url, data=xmlReq)
xmlResp = BeautifulSoup(resp.text)

In [24]:
# Parse response

In [25]:
# Create data visualisation